# Hydrological Data Colection Center
***

**_Autor:_** Chus Casado Rodríguez<br>
**_Fecha:_** 19-05-2025<br>

**Introduction:**<br>
This code downloads the reservoir time series available in the HYDRO data base in Spain.

Use the code from the repository [EFAS_HYDRO](https://github.com/casadoj/efas_hydro/blob/main/notebook/tutorial.ipynb).